In [1]:
import geopandas as gpd
import pandas as pd
import os,re,json
import numpy as np
from p_tqdm import p_map

In [2]:

# pip install mbutil

In [21]:
# lets generate the overview pictures 
#
floc = "tmp/county.geojson"
gpd.read_file('data/CountiesUA.geojson')[['OBJECTID', 'CTYUA20CD', 'CTYUA20NM','LONG', 'LAT', 'geometry']].to_file(floc, driver="GeoJSON")

os.system('tippecanoe -Z0 -z6 -f -o tiles/countries.mbtiles'+ floc)


tippecanoe: tiles/countries.mbtilestmp/county.geojson: unable to open database file


256

In [30]:
 gpd.read_file('data/centroids2001.geojson').columns
    


Index(['OBJECTID', 'OA01CD', 'OA01CDOLD', 'geometry'], dtype='object')

In [32]:
year = '01'

base  = gpd.read_file('data/OA2001.geojson')[['OA01CD', 'geometry']]
centroids = gpd.read_file('data/centroids2001.geojson')[['OA01CD','geometry']]

base.to_file("tmp/OA%s.geojson"%year, driver="GeoJSON")
centroids.to_file("tmp/centroids%s.geojson"%year, driver="GeoJSON")

start = 6
end = 12
# cant have centroids being grouped, but fine for area shapes
os.system('tippecanoe -Z%d -z%d -f -o tiles/centroids%s.mbtiles  ./tmp/centroids%s.geojson'%(start,end,year,year))
os.system('tippecanoe -Z%d -z%d -f -o tiles/outputarea%s.mbtiles --drop-densest-as-needed --generate-id ./tmp/OA%s.geojson'%(start,end,year,year))

os.system('tile-join -o ./tiles/area_centroids_6_9.mbtiles -f ./tiles/centroids%s.mbtiles ./tiles/outputarea%s.mbtiles'%(year,year))

# --generate-id
# --coalesce-densest-as-needed

# tippecanoe -o tiles/test.mbtiles -Z9 -z10 -f ./data/tile_OA11.geojson ./data/tile_centroids11.geojson


# gdf_base = gdf_base.append(df,ignore_index=True)
# gdf_base .to_file("myfile.gpkg", layer="layer1", driver="GPKG")

For layer 0, using name "centroids01"
175434 features, 3178644 bytes of geometry, 16 bytes of separate metadata, 1929782 bytes of string pool
  99.9%  12/2027/1319  
For layer 0, using name "OA01"
175434 features, 50384833 bytes of geometry, 1041 bytes of separate metadata, 1929782 bytes of string pool
tile 6/32/21 size is 533433 with detail 12, >500000    
Going to try keeping the sparsest 84.36% of the features to make it fit
tile 6/31/20 size is 1590858 with detail 12, >500000    
Going to try keeping the sparsest 28.29% of the features to make it fit
tile 6/31/21 size is 1658288 with detail 12, >500000    
Going to try keeping the sparsest 27.14% of the features to make it fit
tile 6/31/20 size is 663640 with detail 12, >500000    
Going to try keeping the sparsest 19.18% of the features to make it fit
tile 6/31/21 size is 720835 with detail 12, >500000    
Going to try keeping the sparsest 16.94% of the features to make it fit
tile 6/31/20 size is 505678 with detail 12, >500000   

0

In [33]:
# base  = gpd.read_file('data/OA2011.geojson')[['OA01CD', 'LAD01CD',
#        'geometry']]
# centroids = gpd.read_file('data/centroids2011.geojson')[['lsoa01cd', 'geometry']]

# base.to_file("tmp/OA11.geojson", driver="GeoJSON")
# centroids.to_file("tmp/centroids11.geojson", driver="GeoJSON")

# os.system('tippecanoe -Z6 -z9 -f -o tiles/area_centroids_6_9.mbtiles --coalesce-densest-as-needed --generate-id ./tmp/OA%d.geojson ./tmp/centroids%d.geojson'%(11,11))




In [34]:
# merge
os.system('tile-join -o ./tiles/merged.mbtiles -f ./tiles/countries.mbtiles ./tiles/area_centroids_6_9.mbtiles')

Tile 6/31/21 size is 515469, >500000. Skipping this tile


0

In [35]:
# tippecanoe -z3 -o countries-z3.mbtiles --coalesce-densest-as-needed ne_10m_admin_0_countries.geojson
# tippecanoe -zg -Z4 -o states-Z4.mbtiles --coalesce-densest-as-needed --extend-zooms-if-still-dropping ne_10m_admin_1_states_provinces.geojson
# tile-join -o states-countries.mbtiles countries-z3.mbtiles states-Z4.mbtiles

In [ ]:
## convert to static server

os.system('rm -rf ./tiles/tileserver; mb-util --image_format=pbf ./tiles/merged.mbtiles ./tiles/tileserver')
os.system('cd ./tiles/tileserver && find . -type f | xargs -n1 -P 1 -t -I % gzip -d -r -S .pbf %')
os.system('cd ./tiles/tileserver && find . -type f | xargs -n1 -P 1 -t -I % mv % %.pbf')


In [ ]:
os.system("git add -A; git commit -m 'recompute tiles'; git push")